In [ ]:
# Install dependencies
!pip install mediapipe tensorflow tensorflow-addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 37.1 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.1
    Uninstalling typeguard-4.4.1:
      Successfully uninstalled typeguard-4.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.4.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


1.Pose Detection and Video Processing
python
Copy code


In [10]:
import cv2
import mediapipe as mp
import numpy as np

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return 360 - angle if angle > 180.0 else angle

video_path = "/content/istockphoto-925697370-640_adpp_is.mp4"
cap = cv2.VideoCapture(video_path)
angles = []

output_path = "output_video.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_resized = cv2.resize(frame, (224, 224))
    image = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
    results = pose.process(image)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
        knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
        ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

        angle = calculate_angle(hip, knee, ankle)
        angles.append(angle)

        frame_height, frame_width, _ = frame.shape
        hip_coords = (int(hip[0] * frame_width), int(hip[1] * frame_height))
        knee_coords = (int(knee[0] * frame_width), int(knee[1] * frame_height))
        ankle_coords = (int(ankle[0] * frame_width), int(ankle[1] * frame_height))

        cv2.line(frame, hip_coords, knee_coords, (255, 0, 0), 2)
        cv2.line(frame, knee_coords, ankle_coords, (255, 0, 0), 2)
        cv2.putText(frame, f'Angle: {int(angle)}', (knee_coords[0] - 50, knee_coords[1] - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

    out.write(frame)

cap.release()
out.release()
pose.close()
print("Knee Angles:", angles)


Knee Angles: [110.73016745264084, 103.41430173777285, 93.86835346289209, 85.17548222919592, 79.41938570133583, 71.8759514606915, 67.50847982647987, 63.95201371486314, 61.202579371712105, 57.76513841931804, 57.065426578236014, 58.70997130434524, 61.59366649098852, 63.460141429284775, 67.92772436884097, 75.28902243272586, 83.17752960547462, 93.305941093763, 100.31034232825465, 106.00254098735694, 111.22353305103684, 117.58306907023821, 124.31136101639547, 130.52921803015067, 138.39701128946197, 133.12931089479653, 156.18835669931542, 158.97341156522597, 162.43079618741913, 165.80175604389925, 168.37156894804326, 169.3356083725421, 170.03689867709323, 168.05944315996186, 140.58457385380518, 172.59865535276526, 169.0887510507849, 178.80047353374624, 177.92273151824435, 176.40652322875047, 177.02460705962943, 175.41258670531425, 175.99135217762293, 176.74493180171228, 174.8004700782369, 158.0913073302454, 109.81585761638105, 102.73636068987406, 93.84174441790006, 84.29727523696877, 79.52928

Mobile Optimization and TensorFlow Lite Conversion

In [11]:
import tensorflow as tf
import numpy as np

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(224, 224, 3)),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.save("pose_model.h5")

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

converter.representative_dataset = lambda: [
    [np.random.rand(1, 224, 224, 3).astype(np.float32)]
]

tflite_model = converter.convert()
with open("pose_detection_model1.tflite", "wb") as f:
    f.write(tflite_model)

print("TensorFlow Lite model saved as pose_detection_model1.tflite.")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Saved artifact at '/tmp/tmpcfc116jx'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_35')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  132323756087552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132323753487600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132323753485136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132323753482848: TensorSpec(shape=(), dtype=tf.resource, name=None)
TensorFlow Lite model saved as pose_detection_model1.tflite.


TensorFlow Lite Inference

In [12]:
import tensorflow as tf
import numpy as np

interpreter = tf.lite.Interpreter(model_path="pose_detection_model1.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']
dummy_input = np.random.rand(*input_shape).astype(np.float32)

interpreter.set_tensor(input_details[0]['index'], dummy_input)
interpreter.invoke()

output_data = interpreter.get_tensor(output_details[0]['index'])
print("Inference output:", output_data)


Inference output: [[0.06195411 0.20173225 0.0774616  0.14309861 0.1125245  0.09503771
  0.078137   0.08052616 0.09552439 0.05400362]]
